In [2]:
import os, json, re, ast
from openai import OpenAI
import concurrent.futures
import threading
from time import sleep
# set the key
API_SECRET_KEY = "sk-zk2277dd97db0e3d8a2600807992f389c5365b8fb3005750"
BASE_URL = "https://flag.smarttrot.com/v1/"
TYPE = "count"

In [3]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def write_to_file(file_path, content):
    with open(file_path, "w") as f:
        f.write(content)

system_message = read_file(os.path.join(TYPE, "system_message.txt"))
example_000_caps = read_file(os.path.join(TYPE, "000_caps.txt"))
example_000_conv = read_file(os.path.join(TYPE, "000_conv.txt"))
example_001_caps = read_file(os.path.join(TYPE, "001_caps.txt"))
example_001_conv = read_file(os.path.join(TYPE, "001_conv.txt"))
result_path = os.path.join(TYPE, "results")
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [4]:
def chat_completions3(query):
    client = OpenAI(api_key=API_SECRET_KEY, base_url=BASE_URL)
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": example_000_caps},
            {"role": "assistant", "content": example_000_conv},
            {"role": "user", "content": example_001_caps},
            {"role": "assistant", "content": example_001_conv},
            {"role": "user", "content": query},
        ],
        temperature=0.3,
    )
    return resp.choices[0].message.content

In [5]:
with open('LEVIR-MCI-dataset/LevirCCcaptions-v2-f.json', 'r') as f:
    datajson = json.load(f)

datajson = [x for x in datajson if x['filepath'] == 'train']

In [8]:
def get_result_conv(row):
    file_path = os.path.join(result_path, row['filename'].replace('.png','.txt'))
    if os.path.exists(file_path):
        return
    query = '\n'.join(['- ' + x.replace(' .', '.').strip() for x in row['sentences']])
    ans = chat_completions3(query)
    write_to_file(file_path, ans)

def get_result_count(row):
    query = "Change captions:\n"
    file_path = os.path.join(result_path, row['filename'].replace('.png','.txt'))
    if os.path.exists(file_path):
        return
    if row['changeflag']==0:
        ans = '''Question: How many roads and buildings have changed between the two images?
Answer: There are no changes in roads or buildings between the two images.'''
        write_to_file(file_path, ans)
    else:
        query += '\n'.join(['- ' + x.replace(' .', '.').strip() for x in row['sentences']])
        query += "\n\n"
        query += "Change counts:\n"
        query += json.dumps(row['change_counts'])
        query += "\n\n"
        query += "Change contours:\n"
        query += json.dumps(row['contours'])
        ans = chat_completions3(query)
        write_to_file(file_path, ans)

In [9]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 并行执行，提高效率
    executor.map(lambda x: get_result_count(x), datajson)